<a href="https://colab.research.google.com/github/SridharK12/Deep-Learning-Project/blob/main/stackoverflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline
import collections

In [ ]:
from google.colab import drive
drive.mount('/content/drive')  

In [ ]:
data = pd.read_csv("/content/drive/My Drive/Project1 Data/Questions.csv",encoding='iso-8859-1')

In [ ]:
questions=data

In [ ]:
answers = pd.read_csv("/content/drive/My Drive/Project1 Data/Answers.csv",encoding='iso-8859-1')

In [ ]:
Tags = pd.read_csv("/content/drive/My Drive/Project1 Data/Tags.csv",encoding='iso-8859-1')

In [ ]:
questions.shape

In [ ]:
answers.shape

In [ ]:
Tags.shape

In [ ]:
questions.head()

In [ ]:
Tags.head()

In [ ]:
answers.head()

In [ ]:
#questions_sample = questions.sample(frac =.1)

In [ ]:
#answers_sample=answers.sample(frac=0.1)

In [ ]:
#Tags_sample=Tags.sample(frac=0.1)

In [ ]:
#tagCount =  collections.Counter(list(Tags_sample['Tag'])).most_common(10)
#print(tagCount)

In [ ]:
tagCount =  collections.Counter(list(Tags['Tag'])).most_common(10)
print(tagCount)

In [ ]:
top10=['javascript','java','c#','php','android','jquery','python','html','c++','ios']

In [ ]:
#tag_top10= Tags[Tags_sample.Tag.isin(top10)]

In [ ]:
tag_top10= Tags[Tags.Tag.isin(top10)]

In [ ]:
len(top10),tag_top10.shape

In [ ]:
tag_top10.head()

In [ ]:
Tags['Tag'].value_counts()

In [ ]:
def add_tags(question_id):
    return tag_top10[tag_top10['Id'] == question_id['Id']].Tag.values

top10 = tag_top10.apply(add_tags, axis=1)

In [ ]:
len(top10),tag_top10.shape

In [ ]:
questions.head()

In [ ]:
top10.head()

In [ ]:
tag_top10=pd.concat([tag_top10, top10.rename('Tags')], axis=1)
tag_top10.head()

In [ ]:
tag_top10.drop(["Tag"], axis=1, inplace=True)
tag_top10.shape

In [ ]:
top10_tags=tag_top10.loc[tag_top10.astype(str).drop_duplicates().index]

In [ ]:
top10_tags

In [ ]:
total=pd.merge(questions, top10_tags, on='Id')
print(total.shape)
total.head()

In [ ]:
#questions_sample.shape

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from nltk import word_tokenize
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, BatchNormalization, GRU ,concatenate

In [ ]:
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(total.Tags)
labels = multilabel_binarizer.classes_
labels

In [ ]:
#train,test=train_test_split(total,test_size=0.25,random_state=24)
train,test=train_test_split(total[:550000],test_size=0.25,random_state=24)

In [ ]:
train.shape,test.shape

In [ ]:
X_train_t=train['Title']
X_train_b=train['Body']
y_train=multilabel_binarizer.transform(train['Tags'])
X_test_t=test['Title']
X_test_b=test['Body']
y_test=multilabel_binarizer.transform(test['Tags'])

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
sent_lens_t=[]
for sent in train['Title']:
    sent_lens_t.append(len(word_tokenize(sent)))
max(sent_lens_t)

In [ ]:
np.quantile(sent_lens_t,0.97)

In [ ]:
max_len_t = 18
tok = Tokenizer(char_level=False,split=' ')
tok.fit_on_texts(X_train_t)
sequences_train_t = tok.texts_to_sequences(X_train_t)

In [ ]:
vocab_len_t=len(tok.index_word.keys())
vocab_len_t

In [ ]:
sequences_matrix_train_t = sequence.pad_sequences(sequences_train_t,maxlen=max_len_t)
sequences_matrix_train_t

In [ ]:
sequences_test_t = tok.texts_to_sequences(X_test_t)
sequences_matrix_test_t = sequence.pad_sequences(sequences_test_t,maxlen=max_len_t)

In [ ]:
sequences_matrix_train_t.shape,sequences_matrix_test_t.shape,y_train.shape,y_test.shape

In [ ]:
sent_lens_b=[]
for sent in train['Body']:
    sent_lens_b.append(len(word_tokenize(sent)))
max(sent_lens_b)

In [ ]:
np.quantile(sent_lens_b,0.90)

In [ ]:
max_len_b = 600
tok = Tokenizer(char_level=False,split=' ')
tok.fit_on_texts(X_train_b)
sequences_train_b = tok.texts_to_sequences(X_train_b)

In [ ]:
vocab_len_b =len(tok.index_word.keys())
vocab_len_b 

In [ ]:
sequences_matrix_train_b = sequence.pad_sequences(sequences_train_b,maxlen=max_len_b)
sequences_matrix_train_b

In [ ]:
sequences_test_b = tok.texts_to_sequences(X_test_b)
sequences_matrix_test_b = sequence.pad_sequences(sequences_test_b,maxlen=max_len_b)

In [ ]:
sequences_matrix_train_t.shape,sequences_matrix_train_b.shape,y_train.shape

In [ ]:
sequences_matrix_test_t.shape,sequences_matrix_test_b.shape,y_test.shape

In [ ]:
#from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, BatchNormalization, GRU ,concatenate
#from keras.models import Model
#from tensorflow.keras.models import Sequential, Model
#def RNN():
#  title_input = Input(name='title_input',shape=[max_len_t])
#  title_Embed = Embedding(vocab_len_t+1,2000,input_length=max_len_t,mask_zero=True,name='title_Embed')(title_input)
#  gru_out_t = GRU(300)(title_Embed)
#  auxiliary_output = Dense(10, activation='sigmoid', name='aux_output')(gru_out_t)   
    
#  body_input = Input(name='body_input',shape=[max_len_b]) 
#  body_Embed = Embedding(vocab_len_b+1,170,input_length=max_len_b,mask_zero=True,name='body_Embed')(body_input)
#  gru_out_b = GRU(200)(body_Embed)

#  com = concatenate([gru_out_t, gru_out_b])

#  dense1 = Dense(400,activation='relu')(com)
#  dp1 = Dropout(0.5)(dense1)
#  bn = BatchNormalization()(dp1) 
#  dense2 = Dense(150,activation='relu')(bn)
    
#  main_output = Dense(10, activation='sigmoid', name='main_output')(dense2)
    
#  model = Model(inputs=[title_input, body_input],outputs=[main_output, auxiliary_output])
#  return model

In [ ]:
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, BatchNormalization, GRU ,concatenate
from keras.models import Model
from tensorflow.keras.models import Sequential, Model

from tensorflow.keras.optimizers import SGD
opt = SGD(lr=0.01)

title_input = Input(name='title_input',shape=[max_len_t])
title_Embed = Embedding(vocab_len_t+1,2000,input_length=max_len_t,mask_zero=True,name='title_Embed')(title_input)
gru_out_t = GRU(300)(title_Embed)
auxiliary_output = Dense(10, activation='sigmoid', name='aux_output')(gru_out_t)   
    
body_input = Input(name='body_input',shape=[max_len_b]) 
body_Embed = Embedding(vocab_len_b+1,170,input_length=max_len_b,mask_zero=True,name='body_Embed')(body_input)
gru_out_b = GRU(200)(body_Embed)

com = concatenate([gru_out_t, gru_out_b])

dense1 = Dense(400,activation='relu')(com)
dp1 = Dropout(0.5)(dense1)
bn = BatchNormalization()(dp1) 
dense2 = Dense(150,activation='relu')(bn)
    
main_output = Dense(10, activation='sigmoid', name='main_output')(dense2)
    
model = Model(inputs=[title_input, body_input],outputs=[main_output, auxiliary_output])

In [ ]:
#model=RNN
model.summary()

In [ ]:
model.compile(optimizer=opt,loss={'main_output': 'categorical_crossentropy', 'aux_output': 'categorical_crossentropy'},
              metrics=['accuracy'])

In [ ]:
results=model.fit({'title_input': sequences_matrix_train_t, 'body_input': sequences_matrix_train_b},
          {'main_output': y_train, 'aux_output': y_train},
          validation_data=[{'title_input': sequences_matrix_test_t, 'body_input': sequences_matrix_test_b},
          {'main_output': y_test, 'aux_output': y_test}],
          epochs=50, batch_size=2000)

Epoch 1/50
 72/207 [=========>....................] - ETA: 39:56 - loss: 5.3195 - main_output_loss: 2.6485 - aux_output_loss: 2.6709 - main_output_accuracy: 0.1681 - aux_output_accuracy: 0.1333

In [ ]:
(predicted_main, predicted_aux)=model.predict({'title_input': sequences_matrix_test_t, 'body_input': sequences_matrix_test_b},verbose=1)


In [ ]:
from sklearn.metrics import classification_report,f1_score

In [ ]:
print(f1_score(y_test,predicted_main>.55,average='samples'))

In [ ]:
print(classification_report(y_test,predicted_main>.55))

In [ ]:
test.iloc[24]

In [ ]:
predicted_main[24].round(decimals = 2)

In [ ]:
labels